# Лабораторная работа 1. Линейная регрессия и PCA

В лабораторной работе изучаем процесс построения регрессионных моделей на примере датасета *diamonds*. Рассматриваем классическую линейную регрессию, гребневую регрессию и сокращение размерности признаков с помощью анализа главных компонент (PCA).

## Введение
Цель работы — освоить базовый цикл работы с моделями линейной регрессии: изучить исходные данные, выполнить предобработку, оценить мультиколлинеарность, сравнить качество моделей с и без регуляризации, а затем проверить, как метод главных компонент влияет на результаты. В работе применяются библиотеки `pandas`, `numpy`, `scikit-learn`, `statsmodels`, `matplotlib` и `seaborn`.

## Описание датасета
Используется набор данных [diamonds](https://www.kaggle.com/datasets/shivam2503/diamonds), который содержит характеристики бриллиантов (вес, качество огранки, цвет, чистота, физические размеры) и их цену. Всего 53 940 наблюдений. Целевая переменная — `price` (стоимость камня). Ниже показаны основные визуализации, подготовленные скриптом `text.py`:

- распределения числовых признаков,
- распределение целевой переменной `price`,
- корреляционная матрица.

![Numeric distributions](figures/numeric_feature_distributions.png)

![Price distribution](figures/price_distribution.png)

![Correlation matrix](figures/correlation_matrix.png)

In [ ]:
from pathlib import Path
import pandas as pd

DATA_PATH = Path('diamonds.csv')

data = pd.read_csv(DATA_PATH).drop(columns=['Unnamed: 0'])
data.head()

In [ ]:
data.describe(include='all')

### Анализ пропусков
В наборе отсутствуют пропущенные значения — таблица ниже показывает нули для всех столбцов.

In [ ]:
data.isna().sum()

## Подготовка данных
Для дальнейшего моделирования выполняются следующие шаги:

- **Разделение выборки.** Используется `train_test_split` c долей тестовой выборки 20 % и фиксированным `random_state=42`.
- **Масштабирование числовых признаков.** Применяется `StandardScaler` внутри `ColumnTransformer`.
- **Кодирование категориальных признаков.** Используется `OneHotEncoder` с отключением ошибок на неизвестных значениях.
- **Мультиколлинеарность.** Рассчитаны коэффициенты VIF для числовых признаков; результаты сохранены в `results/vif_values.csv`.

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.model_selection import KFold, cross_validate
from sklearn.decomposition import PCA
from statsmodels.stats.outliers_influence import variance_inflation_factor

RANDOM_STATE = 42
TEST_SIZE = 0.2

numeric_features = ['carat', 'depth', 'table', 'x', 'y', 'z']
categorical_features = ['cut', 'color', 'clarity']

X = data.drop(columns=['price'])
y = data['price']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
)

numeric_pipeline = Pipeline([('scaler', StandardScaler())])
categorical_pipeline = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer([
    ('num', numeric_pipeline, numeric_features),
    ('cat', categorical_pipeline, categorical_features)
], remainder='drop', sparse_threshold=0)

vif_df = pd.DataFrame({
    'feature': numeric_features,
    'vif': [variance_inflation_factor(
        data[numeric_features].replace(0, 1e-6).values, i
    ) for i in range(len(numeric_features))]
})
vif_df

## Ход работы
Сначала обучаем две модели на исходных признаках:

1. Классическая линейная регрессия.
2. Гребневая регрессия (`alpha = 10`).

Обе модели оцениваются на тестовой выборке (RMSE, R², MAPE) и через 5-кратную кросс-валидацию. Результаты загружаем из `results/metrics_original_features.csv`.

In [ ]:
metrics_original = pd.read_csv('results/metrics_original_features.csv')
metrics_original

Далее снижаем мультиколлинеарность с помощью PCA. Перед понижением размерности используем такие же шаги предобработки, затем подбираем число компонент, сохраняя не менее 95 % дисперсии. График метода локтя (накопленная объяснённая дисперсия) показан ниже.

![PCA explained variance](figures/pca_explained_variance.png)

На полученных главных компонентах повторяем обучение линейной и гребневой регрессии.

In [ ]:
metrics_pca = pd.read_csv('results/metrics_pca_features.csv')
metrics_pca

### Сводное сравнение моделей
Ниже представлена объединённая таблица метрик для моделей на исходных признаках и на PCA-компонентах.

In [ ]:
metrics_comparison = pd.read_csv('results/metrics_comparison.csv')
metrics_comparison

## Заключение
- Исходный датасет не содержит пропусков, а признаки товара сильно коррелируют между собой (особенно размеры и вес). VIF показывает высокую мультиколлинеарность для `carat`, `x`, `y`, `z`.
- Линейная и гребневая регрессии на исходных данных демонстрируют близкое качество, при этом гребневая обеспечивает более устойчивые результаты на кросс-валидации.
- После применения PCA (с сохранением 95 % дисперсии) качество моделей остаётся сопоставимым: небольшое снижение RMSE компенсируется более стабильными метриками и устранением мультиколлинеарности.
- Метод PCA помогает упростить модель без заметной потери точности и облегчает интерпретацию за счёт устранения коррелированных признаков.